In [1]:
import pandas as pd
import numpy as np
import requests
import tweepy
import json
import os
import re
from IPython.display import clear_output

------
<a id="indice"></a>

# Índice

1. **[Coleta](#coleta)**
    * [Twitter Archive](#coleta:twitter-archive)
    * [Image Prediction](#coleta:image-prediction)
    * [Twitter API](#coleta:twitter-api)
    
    
2. **[Avaliação](#avaliacao)**
    * [Twitter Archive](#avaliacao:twitter-archive)
    * [Image Prediction](#avaliacao:image-prediction)
    * [Twitter API](#avaliacao:twitter-api)
    * [Notas da avaliação](#avaliacao:anotacoes)
    
    
3. **[Limpeza](#limpeza)**
    * [Definição](#limpeza:definicao)
    * [Twitter Archive](#limpeza:twitter-archive)
    * [Image Prediction](#limpeza:image-prediction)
    * [Twitter API](#limpeza:twitter-api)


4. **[Armazenamento](#armazenamento)**

------
<a id="coleta"></a>

# Coleta

<a id="coleta:twitter-archive"></a>

## Coleta: Twitter archive

In [2]:
df_twitter_arc = pd.read_csv('data/twitter-archive-enhanced.csv')
df_twitter_arc.sample(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1845,675845657354215424,NaN,NaN,2015-12-13 01:12:15 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Pepper. She's not fully comfortable ri...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/675845657...,10,10,Pepper,None,None,pupper,None
1352,703774238772166656,NaN,NaN,2016-02-28 02:50:28 +0000,"<a href=""http://twitter.com/download/iphone"" r...","""YOU CAN'T HANDLE THE TRUTH"" both 10/10 https:...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/703774238...,10,10,None,None,None,None,None


<a id="coleta:image-prediction"></a>

## Coleta: Image prediction

In [3]:
df_prediction = None

r = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')

if r.status_code is 200:
    df_prediction = pd.read_csv(pd.compat.StringIO(r.text), sep='\t')    
else:    
    print('ERROR: Image prediction request returned {status_code} status code.'.format(status_code = r.status_code))

In [4]:
df_prediction.sample(2)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
2065,890240255349198849,https://pbs.twimg.com/media/DFrEyVuW0AAO3t9.jpg,1,Pembroke,0.511319,True,Cardigan,0.451038,True,Chihuahua,0.029248,True
84,667502640335572993,https://pbs.twimg.com/media/CUNyHTMUYAAQVch.jpg,1,Labrador_retriever,0.996709,True,golden_retriever,0.001688,True,beagle,0.000712,True


<a id="coleta:twitter-api"></a>

## Coleta: Twitter API

In [5]:
# Carregar configuração do Twitter App
with open('twitter_config.json', 'r', encoding='utf-8') as file:
    app_config = json.load(file)

In [6]:
# Atribuição da configurações a variaveis locais
api_key = app_config['api_key']
api_secret = app_config['api_secret']
access_token = app_config['access_token']
access_secret = app_config['access_secret']

In [7]:
# Autentificação ao Twitter App
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

Buscar dados do histórico de Tweets na API

In [8]:
json_path = 'data/tweet_json.txt'
error_log = 'data/tweet_error.log'

# Salvar Tweets localmente
if(not os.path.isfile(json_path)):
    print('Aguarde. Dado o volume de requisições, a coleta de tweets pode demorar alguns minutos.')
    tweet_count = 0
    for tweet_id in df_twitter_arc.tweet_id:
        print('{percent}%'.format(percent=int((++tweet_count/df_twitter_arc.tweet_id.size)*100)))
        try:
            status = api.get_status(tweet_id)
            with open(json_path, 'a', newline='\n') as file:
                file.write(f'{json.dumps(status._json)}\n')

        except Exception as err:
            with open(error_log, 'a', newline='\n') as log:
                log.write(f'{str(tweet_id)}: {str(err.args[0])}\n')
            print(f'{str(tweet_id)}: {str(err.args[0])}')
        
        clear_output(wait=True)
else:
    print('Dados já salvos em disco, não serão executadas novas requisições à API do Twitter.')
        

Dados já salvos em disco, não serão executadas novas requisições à API do Twitter.


In [9]:
# Criar uma lista de Dictionaries com os Tweets carregados
tweets = []

with open(json_path, 'r') as file:
    for line in file:
        try:
            tweet = json.loads(line)
            
            if(tweet.get('entities', False)):
                if(tweet['entities'].get('media', False) and tweet['entities']['media'][0].get('media_url', False)):
                    tweets.append({
                        'id': int(tweet['id']),
                        'created_at': tweet['created_at'],
                        'in_reply_to_status_id': str(tweet['in_reply_to_status_id']),
                        'in_reply_to_user_id': str(tweet['in_reply_to_user_id']),
                        'is_quote_status': tweet['is_quote_status'],
                        'retweet_count': int(tweet['retweet_count']),
                        'favorite_count': int(tweet['favorite_count']),
                        'media_url': tweet['entities']['media'][0]['media_url'],
                        'retweeted': tweet['retweeted'],
                        'favorited': tweet['favorited']
                    })
            
            
        except Exception as e:
            print(e) 
            
        

In [10]:
# Criar dataframe de Tweets consultados na API
columns = tweets[0].keys()
df_tweets_api = pd.DataFrame(tweets, columns = columns)

------
<a id="avaliacao"></a>

# Avaliação

<a id="avaliacao:twitter-archive"></a>

## Avaliação: Twitter data archive

In [11]:
 df_twitter_arc.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
186,856288084350160898,8.562860e+17,279280991.0,2017-04-23 23:26:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@xianmcguire @Jenna_Marbles Kardashians wouldn...,NaN,NaN,NaN,NaN,14,10,None,None,None,None,None
1802,676949632774234114,NaN,NaN,2015-12-16 02:19:04 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tyrus. He's a Speckled Centennial Tico...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/676949632...,8,10,Tyrus,None,None,None,None
134,866686824827068416,NaN,NaN,2017-05-22 16:06:55 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Lili. She can't believe you betrayed h...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/866686824...,12,10,Lili,None,None,None,None
2338,666104133288665088,NaN,NaN,2015-11-16 04:02:55 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Not familiar with this breed. No tail (weird)....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666104133...,1,10,None,None,None,None,None
2319,666447344410484738,NaN,NaN,2015-11-17 02:46:43 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Scout. She is a black Downton Abbey. I...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666447344...,9,10,Scout,None,None,None,None


In [12]:
df_twitter_arc.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [13]:
df_twitter_arc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

<a id="twitter-arc:info"></a>
* Dados incompletos, contém apenas 2356 registros dos 5000 anunciados
* `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id` e `retweeted_status_user_id` em `float`
* `timestamp` e `retweeted_status_timestamp` em `string`

In [14]:
# Verificar se há IDs duplicados
df_twitter_arc[df_twitter_arc.tweet_id.duplicated()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


<a id="twitter-arc:source"></a>

In [15]:
# Visualizar exemplos de dados da coluna `source`
df_twitter_arc.loc[:, 'source'].sample(10)

2269    <a href="http://twitter.com" rel="nofollow">Tw...
1403    <a href="http://twitter.com/download/iphone" r...
105     <a href="http://twitter.com/download/iphone" r...
915     <a href="http://twitter.com/download/iphone" r...
1073    <a href="http://twitter.com/download/iphone" r...
448     <a href="http://twitter.com/download/iphone" r...
476     <a href="http://twitter.com/download/iphone" r...
1669    <a href="http://twitter.com/download/iphone" r...
921     <a href="http://twitter.com/download/iphone" r...
429     <a href="http://twitter.com/download/iphone" r...
Name: source, dtype: object

Variável `source` é uma URL para a Aplicação utilizada no tweet, sem valor para a unidade de observação

<a id="twitter-arc:name"></a>

In [16]:
# Visualizar exemplos da coluna `name`
df_twitter_arc.name.sample(10)

1610       None
2126    Butters
1510     Bailey
298        None
1291     Cooper
280     Winston
2140    Raphael
576        None
2004      Franq
352        None
Name: name, dtype: object

In [17]:
df_twitter_arc[df_twitter_arc.name == 'None'].name.count()

745

* Nomes (`name`) preenchidos com valor literal igual a `'None'`, supostamente um valor não preenchido, vazio

<a id="twitter-arc:category"></id>

In [18]:
# Visualizar exemplo de dados das colunas 'doggo' a 'puppo'
df_twitter_arc.loc[:,'doggo':].sample(10)

,doggo,floofer,pupper,puppo
2074,None,None,None,None
1043,None,None,None,None
1209,None,None,None,None
254,None,None,None,None
317,None,None,None,None
136,None,None,None,None
2177,None,None,None,None
2204,None,None,None,None
1807,None,None,pupper,None
431,None,None,None,None


* Valores não preenchidos como 'None' literal

Uma variável em várias colunas

In [19]:
# Verificar padrão no preenchimento das classificações
df_twitter_arc.loc[:,'doggo':].nunique()

doggo      2
floofer    2
pupper     2
puppo      2
dtype: int64

In [20]:
df_twitter_arc.loc[:, 'doggo':'puppo'].isna().sum()

doggo      0
floofer    0
pupper     0
puppo      0
dtype: int64

<a id="twitter-arc:expanded_urls"></a>

Exemplos da coluna `expanded_urls`

In [21]:
df_twitter_arc.loc[:, 'expanded_urls'].sample(20)

1882    https://twitter.com/dog_rates/status/674999807...
1135    https://twitter.com/dog_rates/status/728409960...
1006                        https://vine.co/v/iqIZFtOxEMB
2106    https://twitter.com/dog_rates/status/670468609...
510     https://twitter.com/dog_rates/status/812372279...
1609    https://twitter.com/dog_rates/status/685547936...
1412    https://twitter.com/dog_rates/status/698953797...
729     https://twitter.com/dog_rates/status/781955203...
32      https://twitter.com/dog_rates/status/886053434...
1697    https://twitter.com/dog_rates/status/681231109...
842     https://twitter.com/dog_rates/status/766793450...
418     https://twitter.com/dog_rates/status/822462944...
1469    https://twitter.com/dog_rates/status/694183373...
447     https://twitter.com/dog_rates/status/819006400...
1970    https://twitter.com/dog_rates/status/673295268...
1931    https://twitter.com/dog_rates/status/674036086...
36      https://twitter.com/dog_rates/status/830583320...
905           

In [22]:
df_twitter_arc.loc[:, 'expanded_urls'].isna().value_counts()

False    2297
True       59
Name: expanded_urls, dtype: int64

* Registros de `expanded_urls` com URLs para os twittes originais. Não apresenta valor a unidade em observacão

------
<a id="avaliacao:image-prediction"></a>

## Avaliação: Image prediction

In [23]:
df_prediction.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


<a id="image-prediction:info"></info>

In [24]:
df_prediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [25]:
df_prediction.sample(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
723,686007916130873345,https://pbs.twimg.com/media/CYUwjz-UAAEcdi8.jpg,1,Rhodesian_ridgeback,0.885301,True,redbone,0.042335,True,seat_belt,0.010493,False
1918,855459453768019968,https://pbs.twimg.com/media/C98z1ZAXsAEIFFn.jpg,2,Blenheim_spaniel,0.389513,True,Pekinese,0.188220,True,Japanese_spaniel,0.082628,True
1986,872486979161796608,https://pbs.twimg.com/media/DBuyRlTUwAAYhG9.jpg,1,Pembroke,0.931861,True,Cardigan,0.037721,True,Chihuahua,0.011967,True
1236,746726898085036033,https://pbs.twimg.com/media/ClzoJz7WYAELHSf.jpg,1,golden_retriever,0.256505,True,Labrador_retriever,0.252417,True,seat_belt,0.203163,False
359,672622327801233409,https://pbs.twimg.com/media/CVWicBbUYAIomjC.jpg,1,golden_retriever,0.952773,True,Labrador_retriever,0.010835,True,clumber,0.008786,True
2056,888554962724278272,https://pbs.twimg.com/media/DFTH_O-UQAACu20.jpg,3,Siberian_husky,0.700377,True,Eskimo_dog,0.166511,True,malamute,0.111411,True
1212,743210557239623680,https://pbs.twimg.com/media/ClBqDuDWkAALK2e.jpg,1,golden_retriever,0.930705,True,Chesapeake_Bay_retriever,0.025934,True,Labrador_retriever,0.007535,True
1848,839549326359670784,https://pbs.twimg.com/media/C6atpTLWYAIL7bU.jpg,1,swing,0.393527,False,Norwich_terrier,0.052480,True,Pembroke,0.049901,True
974,706901761596989440,https://pbs.twimg.com/media/Cc9rZlBWwAA56Ra.jpg,1,wild_boar,0.859499,False,hog,0.128981,False,warthog,0.011318,False
201,669682095984410625,https://pbs.twimg.com/media/CUswUBRUAAAahAo.jpg,1,Christmas_stocking,0.188397,False,studio_couch,0.086887,False,bookcase,0.082599,False


<a id="image-prediction:p-values"></a>

Valores não padronizados para as variáveis `p1`, `p2` e `p3`

In [26]:
# Visualização de exemplos de valores nas colunas `p1`, `p2` e `p3`
df_prediction.loc[:, ['p1', 'p2', 'p3']].sample(5)

,p1,p2,p3
1412,Bernese_mountain_dog,Greater_Swiss_Mountain_dog,EntleBucher
1975,comic_book,envelope,book_jacket
282,Shih-Tzu,golden_retriever,Labrador_retriever
1356,Samoyed,malamute,West_Highland_white_terrier
185,Labrador_retriever,German_short-haired_pointer,Chesapeake_Bay_retriever


<a id="image-prediction:duplicated-urls"></a>

Verificar a duplicidade de Tweet ID (`tweet_id`) e Imagens (`jpg_url`)

In [27]:
# Verificar a duplicidade de IDs
df_prediction.tweet_id.nunique()

2075

Verificar a duplicidade na coluna `jpg_url`

In [28]:
df_prediction.jpg_url.duplicated().value_counts()

False    2009
True       66
Name: jpg_url, dtype: int64

Há 66 imagens duplicadas

In [29]:
# Validar indicadores de confiabiliade da predição
df_prediction.loc[:, ['p1_conf', 'p2_conf', 'p3_conf']].max()

p1_conf    1.000000
p2_conf    0.488014
p3_conf    0.273419
dtype: float64

------
<a id="avaliacao:twitter-api"></a>

## Avaliação: Twitter API Requests

In [30]:
df_tweets_api.describe()

,id,retweet_count,favorite_count
count,1.820000e+03,1820.000000,1820.000000
mean,7.237270e+17,2512.144505,6820.872527
std,5.777841e+16,4891.076169,11900.984651
min,6.660209e+17,11.000000,0.000000
25%,6.747671e+17,536.000000,1363.500000
50%,7.008223e+17,1118.000000,3153.000000
75%,7.617466e+17,2526.250000,7463.250000
max,8.924206e+17,82905.000000,163034.000000


<a id="tweets-api:info"></a>

In [31]:
df_tweets_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1820 entries, 0 to 1819
Data columns (total 10 columns):
id                       1820 non-null int64
created_at               1820 non-null object
in_reply_to_status_id    1820 non-null object
in_reply_to_user_id      1820 non-null object
is_quote_status          1820 non-null bool
retweet_count            1820 non-null int64
favorite_count           1820 non-null int64
media_url                1820 non-null object
retweeted                1820 non-null bool
favorited                1820 non-null bool
dtypes: bool(3), int64(3), object(4)
memory usage: 104.9+ KB


Os nomes das colunas `id` e `media_url` estão fora do padrão dos demais dataframes, respectivamente `tweet_id` e `jpg_url` 

In [32]:
df_tweets_api.sample(5)

,id,created_at,in_reply_to_status_id,in_reply_to_user_id,is_quote_status,retweet_count,favorite_count,media_url,retweeted,favorited
704,725786712245440512,Thu Apr 28 20:40:11 +0000 2016,None,None,False,1431,4311,http://pbs.twimg.com/media/ChKDKmIWIAIJP_e.jpg,False,False
1084,687102708889812993,Wed Jan 13 02:43:46 +0000 2016,None,None,False,1044,2433,http://pbs.twimg.com/media/CYkURJjW8AEamoI.jpg,False,False
913,700505138482569216,Fri Feb 19 02:20:14 +0000 2016,None,None,False,599,2377,http://pbs.twimg.com/media/Cbixs3vUUAAqHHN.jpg,False,False
1760,667119796878725120,Wed Nov 18 23:18:48 +0000 2015,None,None,False,127,327,http://pbs.twimg.com/media/CUIV6F7XIAA1tAM.jpg,False,False
13,885528943205470208,Thu Jul 13 15:58:47 +0000 2017,None,None,False,6230,35088,http://pbs.twimg.com/media/DEoH3yvXgAAzQtS.jpg,False,False


* `created_at` como `string`
* `in_reply_to_status_id` e `in_reply_to_userid` como `string`

<a id="tweets-api:duplicated"></a>
Verificar a duplicidade de valores

In [33]:
# Verificar se há IDs duplicados
df_tweets_api.id.duplicated().value_counts()

False    1820
Name: id, dtype: int64

In [34]:
# Verificar se há imagens duplicadas
df_tweets_api.media_url.duplicated().value_counts()

False    1759
True       61
Name: media_url, dtype: int64

Há tweets referenciando a mesma imagem

------

<a id="avaliacao:anotacoes"></a>

## Notas da avaliação
### Qualidade

#### `df_twitter_arc` - Twitter archive
1. [Base incompleta, contém apenas **2356** registros dos **5000** anunciados](#twitter-arc:info)
* [Coluna `source` sem valor à unidade de obsevação](#twitter-arc:source)
* [Nomes (`name`) preenchidos com valor literal igual a `'None'`, vazio](#twitter-arc:name)
* [Coluna `expanded_urls` sem valor à unidade de observação](#twitter-arc:expanded_urls)
* [Categorias \[`doggo`, `floofer`, `pupper`, `puppo`\] com literal `'None'`](#twitter-arc:category)


#### `df_prediction` - Image prediction
1. [Previsões \[`p1`, `p2`, `p3`\] com nomes não padronizados](#image-prediction:p-values)
* [Duplicidade na coluna `jgp_url`](#image-prediction:duplicated-urls)


#### `df_tweets_api` - Twitter API
1. [Diferentes Tweets referenciando a mesma imagem, duplicidade](#tweets-api:duplicated)


### Organização

#### `df_twitter_arc` - Twitter archive
1. [As colunas `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` estão em `float64`](#twitter-arc:info)
* [As colunas `timestamp` e `retweet_status_timestamp` estão em `string`](#twitter-arc:info)
* [Uma variável em várias colunas, `doggo`, `floofer`, `pupper` e `puppo`](#twitter-arc:category)


#### `df_prediction` - Image prediction
1. [Colunas como variáveis na identificação das predições](#image-prediction:info)


#### `df_tweets_api` - Twitter API
1. [Os nomes das colunas `id` e `media_url` estão fora do padrão dos demais dataframes, respectivamente `tweet_id` e `jpg_url` ](#tweets-api:info)
* [Data de criação (`created_at`) como `string`](#tweets-api:info)
* [As colunas `in_reply_to_status_id` e `in_reply_to_user_id` em `string`](#tweets-api:info)

------

<a id="limpeza"></a>

# Limpeza

<a id="limpeza:definicao"></a>

## Limpeza: Definição 

#### `df_twitter_arc` - Twitter archive
1. [Base incompleta, de **5000**, apenas **2356** estão disponíveis](#limpeza:twitter-arc:completeness)
    * OK - Nada a ser feito
* [Coluna `source` sem valor à unidade de observação](#limpeza:twitter-arc:source)
    * O conteúdo apresenta um link para download do App Twitter, portanto não agrega valor à unidade de observação, a coluna pode ser removida
* [Nomes (`name`) não preenchidos com valor literal igual a `'None'`](#limpeza:twitter-arc:name)
    * Alterar valores como 'None' para `np.nan`
* [Coluna `expanded_urls` sem valor à unidade de observação](#limpeza:twitter-arc:expanded_urls)(#twitter-arc:expanded_urls)
    * Não apresentam valor à unidade de observação, são URLs para os tweets em questão. Remover a coluna `expanded_urls`.
* [Categorias \[doggo, floofer, pupper, puppo\] com literal `'None'`](#limpeza:twitter-arc:category)
    * Aplicar valor `np.nan` para nomes iguais a 'None'   
* [As colunas `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` estão em `float64`](#limpeza:twitter-arc:info)
    * Linhas com valor para as colunas `retweeted_status_id` ou `retweeted_status_user_id` podem ser descartadas por indicarem retweet, o objetivo é analisar somente tweets originais
    * Após a remoção retweets, as colunas `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp`, podem ser descartadas
    * As colunas `in_reply_to_status_id` e `in_reply_to_user_id` serão retratadas no dataframe `df_tweets_api`
* [As colunas `timestamp` e `retweet_status_timestamp` estão em `string` e deveriam ser do tipo `datetime`](#limpeza:twitter-arc:datetime)
    * Converter valores para `datetime`
* [Uma variável em várias colunas, `doggo`, `floofer`, `pupper` e `puppo`](#limpeza:twitter-arc:category-merge)
    * Cada variável em apenas uma coluna,  `category`


#### `df_prediction` - Image prediction
1. [Previsões \[`p1`, `p2`, `p3`\] com nomes não padronizados](#limpeza:image-prediction:names)
    * Converter os valores para 'lower case' e substituir '_' por espaços
* [Previsões repetidas sobre a mesma imagem](#limpeza:image-prediction:duplicated-urls)
    * Descartar predições sobre imagens repetidas
* [Colunas como variáveis na identificação das predições](#limpeza:image-prediction:info)
    * Transpor as variáveis nas colunas para linhas


#### `df_tweets_api` - Twitter API
1. [Os nomes das colunas `id` e `media_url` estão fora do padrão dos demais dataframes, respectivamente `tweet_id` e `jpg_url` ](#limpeza:tweets-api:columns)
* [Diferentes Tweets referenciando a mesma imagem, duplicidade](#limpeza:tweets-api:duplicated)
* [Data de criação (`created_at`) como `string`](#limpeza:tweets-api:datetime)
    * Converter datas em string para o formato `datetime`
* [Valor do id na coluna `in_reply_to_status_id` e `in_reply_to_user_id` como `string`](#limpeza:tweets-api:info)
    * Converter para Inteiro as colunas `in_reply_to_status_id` e `in_reply_to_user_id` e atribuir np.nan as que estiverem sem valor

<a id="limpeza:twitter-archive"></a>

## Limpeza: Twitter archive

In [35]:
# Criar cópia do dataframe para a limpeza
df_arch_clean = df_twitter_arc.copy()

<a id="limpeza:twitter-arc:completeness"></a>

### 1. [Base incompleta, de 5000, apenas 2356 estão disponíveis](#limpeza:twitter-arc:completeness)
Nada a ser feito

In [36]:
df_arch_clean.shape

(2356, 17)

<a id="limpeza:twitter-arc:source"></a>

### 2. [Coluna `source` sem valor à unidade de observação](#limpeza:twitter-arc:source)
* O conteúdo apresenta um link para download do App Twitter, portanto não agrega valor à unidade de observação, a coluna pode ser removida

In [37]:
# Remoção da coluna `source`
df_arch_clean.drop('source', axis='columns', inplace=True)

#### Teste: Remoção coluna `source`

In [38]:
# Conferir que a coluna `source` foi removida
df_arch_clean.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'doggo', 'floofer', 'pupper', 'puppo'],
      dtype='object')

<a id="limpeza:twitter-arc:name"></a>

### 3. [Nomes (`name`) não preenchidos com valor literal igual a `'None'`](#limpeza:twitter-arc:name)
    * Alterar valores como 'None' para `np.nan`

In [39]:
# Contagem de nomes iguais a 'None'
df_arch_clean[df_arch_clean.name.str.lower() == 'none'].name.count()

745

In [40]:
# Função para substituição de valores literais 'None' por np.nan
def set_nan_at_none(value):
    if str(value).lower() == 'none':
        return np.nan
    else:
        return value

In [41]:
# Aplicar `np.nan` aos nomes iguais a 'None'
df_arch_clean.name = df_arch_clean.name.apply(set_nan_at_none)

#### Teste: Aplicação de `np.nan` aos nomes como 'None'

In [42]:
df_arch_clean.name.isnull().sum()

745

In [43]:
df_arch_clean[df_arch_clean.name.str.lower() == 'none'].name.count()

0

In [44]:
df_arch_clean.name.sample(10)

1509         Leo
392          Ike
1000         NaN
1870         NaN
1093         NaN
1199    Jennifur
1307    Finnegus
1338       Reese
222       Gidget
1171         NaN
Name: name, dtype: object

<a id="limpeza:twitter-arc:expanded_urls"></a>

### 4. [Coluna `expanded_urls` sem valor à unidade de observação](#limpeza:twitter-arc:expanded_urls)
* Não apresentam valor à unidade de observação, são URLs para os tweets em questão. Remover a coluna `expanded_urls`

In [45]:
df_arch_clean.drop('expanded_urls', axis='columns', inplace=True)

#### Teste: Remoção da coluna `expanded_urls`

In [46]:
df_arch_clean.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'rating_numerator', 'rating_denominator',
       'name', 'doggo', 'floofer', 'pupper', 'puppo'],
      dtype='object')

<a id="limpeza:twitter-arc:category"></a>

### 5. [Categorias \[`doggo`, `floofer`, `pupper`, `puppo`\] com literal `'None'`](#limpeza:twitter-arc:category)
* Aplicar valor `np.nan` para nomes iguais a 'None'   

In [47]:
# Trocar valores 'None' para np.nan nas colunas `doggo`, `floofer`, `pupper`, `puppo`
df_arch_clean.doggo = df_arch_clean.doggo.apply(set_nan_at_none)
df_arch_clean.doggo.isna().value_counts()

True     2259
False      97
Name: doggo, dtype: int64

In [48]:
df_arch_clean.floofer = df_arch_clean.floofer.apply(set_nan_at_none)
df_arch_clean.floofer.isna().value_counts()

True     2346
False      10
Name: floofer, dtype: int64

In [49]:
df_arch_clean.pupper = df_arch_clean.pupper.apply(set_nan_at_none)
df_arch_clean.pupper.isna().value_counts()

True     2099
False     257
Name: pupper, dtype: int64

In [50]:
df_arch_clean.puppo = df_arch_clean.puppo.apply(set_nan_at_none)
df_arch_clean.puppo.isna().value_counts()

True     2326
False      30
Name: puppo, dtype: int64

In [51]:
# Soma de valores preenchidos nas classificações
sum_categories = 0
sum_categories += df_arch_clean.doggo.notna().sum()
sum_categories += df_arch_clean.floofer.notna().sum()
sum_categories += df_arch_clean.pupper.notna().sum()
sum_categories += df_arch_clean.puppo.notna().sum()

sum_categories

394

Apenas 394 dos 2346 registros foram classificados

<a id="limpeza:twitter-arc:info"></a>

### 6. [As colunas `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` estão em `float64`](#limpeza:twitter-arc:info)
* Linhas com algum valor para as colunas `retweeted_status_id` ou `retweeted_status_user_id` podem ser descartadas por indicarem retweet, e o objetivo é analisar somente tweets originais
* Após a remoção retweets, as colunas `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp`, podem ser descartadas
* As colunas `in_reply_to_status_id` e `in_reply_to_user_id` serão retratadas no dataframe `df_tweets_api`

#### Identificação dos tweets originais
df_arch_clean.loc[:,'retweeted_status_id'].isna().value_counts()

In [52]:
df_arch_clean.retweeted_status_id.size

2356

In [53]:
df_arch_clean.loc[:,'retweeted_status_id'].isna().value_counts()

True     2175
False     181
Name: retweeted_status_id, dtype: int64

Dos 2356 registros, 181 são retweets

In [54]:
# Remoção dos retweets
df_arch_clean.drop(df_arch_clean[df_arch_clean.retweeted_status_id.notna()].index, axis=0, inplace=True)

#### Teste: Remoção de Retweets

In [55]:
# Consultar a existência de colunas com retweet_status_preenchido
df_arch_clean.loc[:,'retweeted_status_id'].isna().value_counts()

True    2175
Name: retweeted_status_id, dtype: int64

Remoção das colunas referentes a retweets

In [56]:
df_arch_clean.drop(['retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'], axis='columns', inplace=True)

#### Teste: Verificação da remoção das colunas referentes a retweets

In [57]:
df_arch_clean.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'text', 'rating_numerator', 'rating_denominator', 'name', 'doggo',
       'floofer', 'pupper', 'puppo'],
      dtype='object')

<a id="limpeza:twitter-arc:datetime"></a>

### 7. [As colunas `timestamp` e `retweet_status_timestamp` estão em `string` e deveriam ser do tipo `datetime`](#limpeza:twitter-arc:datetime)
* Converter valores para `datetime`

Haja vista que a coluna `retweet_status_timestamp` foi removida no passo anterior, não será necessária a sua conversão

In [58]:
df_arch_clean.loc[:, ['timestamp']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 1 columns):
timestamp    2175 non-null object
dtypes: object(1)
memory usage: 34.0+ KB


In [59]:
df_arch_clean.timestamp = pd.to_datetime(df_arch_clean.timestamp)

#### Teste: Veriificação da conversão de `string` para `datetime`

In [60]:
df_arch_clean.loc[:, ['timestamp']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 1 columns):
timestamp    2175 non-null datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1)
memory usage: 34.0 KB


<a id="limpeza:twitter-arc:category-merge"></a>

### 8. [Uma variável em várias colunas, `doggo`, `floofer`, `pupper` e `puppo`](#limpeza:twitter-arc:category-merge)
* Cada variável em apenas uma coluna, reduzi-las a coluna `category`

In [61]:
df_arch_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 12 columns):
tweet_id                 2175 non-null int64
in_reply_to_status_id    78 non-null float64
in_reply_to_user_id      78 non-null float64
timestamp                2175 non-null datetime64[ns, UTC]
text                     2175 non-null object
rating_numerator         2175 non-null int64
rating_denominator       2175 non-null int64
name                     1495 non-null object
doggo                    87 non-null object
floofer                  10 non-null object
pupper                   234 non-null object
puppo                    25 non-null object
dtypes: datetime64[ns, UTC](1), float64(2), int64(3), object(6)
memory usage: 220.9+ KB


In [62]:
df_arch_clean.loc[:, 'doggo':].notna().sum()

doggo       87
floofer     10
pupper     234
puppo       25
dtype: int64

In [63]:
df_arch_clean.loc[:, 'doggo':].notna().sum().sum()

356

De **2175** registros, apenas **356** estão classificados

Criar uma lista das categorias para mesclar as colunas, considerando a coluna a esquerda como prioritaria sobre valores nulos a direita.

In [64]:
category = df_arch_clean.doggo
category = category.combine_first(df_arch_clean.floofer)
category = category.combine_first(df_arch_clean.pupper)
category = category.combine_first(df_arch_clean.puppo)

df_arch_clean['category'] = category

In [65]:
df_arch_clean.category.notna().sum()

344

Após a unificação das colunas de classificação, pode-se ver que resultou em apenas **344** registros, diferente dos **356** iniciais. O que indica que haviam registros com mais de uma classificação.

In [66]:
df_arch_clean.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis='columns', inplace=True)

#### Teste: Verificar a criação da coluna `category` a remoção das colunas `doggo`, `floofer`, `pupper` e `puppo`. 

In [67]:
df_arch_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 9 columns):
tweet_id                 2175 non-null int64
in_reply_to_status_id    78 non-null float64
in_reply_to_user_id      78 non-null float64
timestamp                2175 non-null datetime64[ns, UTC]
text                     2175 non-null object
rating_numerator         2175 non-null int64
rating_denominator       2175 non-null int64
name                     1495 non-null object
category                 344 non-null object
dtypes: datetime64[ns, UTC](1), float64(2), int64(3), object(3)
memory usage: 169.9+ KB


------
<a id="limpeza:image-prediction"></a>

## Limpeza: Image prediction 

In [68]:
# Cópia do dataframe para limpeza
df_predict_clean = df_prediction.copy()

<a id="limpeza:image-prediction:names"></a>

### 1. [Previsões \[`p1`, `p2`, `p3`\] com nomes não padronizados](#limpeza:image-prediction:names)
* Converter os valores para 'lower case' e substituir '_' por espaços

In [69]:
clean_prediction_name = lambda value: str(value).lower().replace('_', ' ')

In [70]:
df_predict_clean.loc[:, ['p1', 'p2', 'p3']].sample(5)

,p1,p2,p3
1408,Italian_greyhound,Mexican_hairless,whippet
1099,Rottweiler,black-and-tan_coonhound,American_black_bear
1541,malamute,jean,keeshond
1770,Pembroke,golden_retriever,Labrador_retriever
419,Eskimo_dog,Norwegian_elkhound,malamute


Aplicar a padronização dos nomes em cada uma das colunas de predição

In [71]:
df_predict_clean.p1 = df_predict_clean.p1.apply(clean_prediction_name)

In [72]:
df_predict_clean.p2 = df_predict_clean.p2.apply(clean_prediction_name)

In [73]:
df_predict_clean.p3 = df_predict_clean.p3.apply(clean_prediction_name)

#### Teste: Verificar padronização dos nomes das predições

In [74]:
df_predict_clean.loc[:, ['p1', 'p2', 'p3']].sample(5)

,p1,p2,p3
712,boston bull,french bulldog,boxer
600,french bulldog,chihuahua,cardigan
1781,hippopotamus,mexican hairless,ice lolly
1433,norwegian elkhound,chesapeake bay retriever,malamute
1738,samoyed,pomeranian,arctic fox


<a id="limpeza:image-prediction:duplicated-urls"></a>

### 2. [Predições repetidas sobre a mesma imagem](#limpeza:image-prediction:duplicated-urls)
* Descartar predições sobre imagens repetidas

In [75]:
df_predict_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [76]:
df_predict_clean.loc[:, ['jpg_url', 'p1', 'p2', 'p3']].duplicated().value_counts()

False    2009
True       66
dtype: int64

Dos **2075** registros, **66** são sobre a mesma imagem. Entretanto, seguindo o objetivo de trabalhar com apenas Tweets originais, além de descartar duplicidade é interessante também descartar as refêrencias a Retweets. Para isso, será feita uma consulta a seleção de Tweets originais, que foi efetuada no passo [6 da limpeza do Twitter Archive](#limpeza:twitter-arc:info)

In [77]:
df_predict_clean = df_predict_clean.loc[df_predict_clean.tweet_id.isin(df_arch_clean.tweet_id)]

#### Teste: Seleção de Tweets originais

In [78]:
df_predict_clean.tweet_id.isin(df_arch_clean.tweet_id).value_counts()

True    1994
Name: tweet_id, dtype: int64

Apenas dos **2075**, **1994** são de Tweets originais

------

Dos Tweets originai ainda é necessário verificar a existência de duplicidade de predições.

In [79]:
df_predict_clean.loc[:, ['jpg_url']].duplicated().value_counts()

False    1994
dtype: int64

Nenhuma duplicidade foi encontrada após a remoção das referências de Retweets

In [80]:
df_predict_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    1994 non-null int64
jpg_url     1994 non-null object
img_num     1994 non-null int64
p1          1994 non-null object
p1_conf     1994 non-null float64
p1_dog      1994 non-null bool
p2          1994 non-null object
p2_conf     1994 non-null float64
p2_dog      1994 non-null bool
p3          1994 non-null object
p3_conf     1994 non-null float64
p3_dog      1994 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 161.6+ KB


<a id="limpeza:image-prediction:info"></a>

### 3. [Colunas como variáveis na identificação das predições](#limpeza:image-prediction:info)
* 3.1. Transpor as variáveis nas colunas para linhas:
    * 3.1.1. [`[p1, p2, p3]`, para `prediction_try`](#limpeza:image-prediction:try)
    * 3.1.2. [`[p1_conf, p2_conf, p3_conf]`, para `confidence`](#limpeza:image-prediction:conf)
    * 3.1.3. [`[p1_dog, p2_dog, p3_dog]`, para `is_a_dog`](#limpeza:image-prediction:dog)
* 3.2. [Unir os dataframes de prediction](#limpeza:image-prediction:merge)

In [81]:
df_predict_clean.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1961,866450705531457537,https://pbs.twimg.com/media/DAZAUfBXcAAG_Nn.jpg,2,french bulldog,0.905334,True,boston bull,0.078060,True,pug,0.001771,True
915,701214700881756160,https://pbs.twimg.com/media/Cbs3DOAXIAAp3Bd.jpg,1,chihuahua,0.615163,True,pembroke,0.159509,True,basenji,0.084466,True
416,674019345211760640,https://pbs.twimg.com/media/CVqZBO8WUAAd931.jpg,1,collie,0.992732,True,borzoi,0.005043,True,shetland sheepdog,0.001725,True
1873,845306882940190720,https://pbs.twimg.com/media/C7siH5DXkAACnDT.jpg,1,irish water spaniel,0.567475,True,labrador retriever,0.169496,True,curly-coated retriever,0.101518,True
1308,753655901052166144,https://pbs.twimg.com/media/CnWGCpdWgAAWZTI.jpg,1,miniature pinscher,0.456092,True,toy terrier,0.153126,True,italian greyhound,0.144147,True


<a id="limpeza:image-prediction:try"></a>

#### 3.1.1. [`[p1, p2, p3]`, para `prediction_try`](#limpeza:image-prediction:try)

In [82]:
# Organizando as variáveis de tentativa de predição e seu palpite
df_predict_try = pd.melt(df_predict_clean,
        id_vars=['tweet_id', 'jpg_url'],
        value_vars=['p1', 'p2', 'p3'],
        var_name=['prediction_try'],
        value_name='prediction'
       )

In [83]:
df_predict_try.sample(5)

,tweet_id,jpg_url,prediction_try,prediction
3432,777684233540206592,https://pbs.twimg.com/media/CsrjryzWgAAZY00.jpg,p2,golden retriever
1400,771014301343748096,https://pbs.twimg.com/media/CrMxZzgWIAQUxzx.jpg,p1,meerkat
2301,671538301157904385,https://pbs.twimg.com/media/CVHIhi2WsAEgdKk.jpg,p2,water bottle
3508,791312159183634433,https://pbs.twimg.com/media/CvtONV4WAAAQ3Rn.jpg,p2,toy terrier
5610,813112105746448384,https://pbs.twimg.com/media/C0jBJZVWQAA2_-X.jpg,p3,basenji


Conversão dos valores da tentativa para número inteiro

In [84]:
# Metodo para retirar o índice da tentativa dos nomes das colunas
get_prediction_index = lambda value: int(re.sub(r'[^\d]', '', str(value)))

In [85]:
df_predict_try.prediction_try = df_predict_try.prediction_try.apply(get_prediction_index)

In [86]:
df_predict_try.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5982 entries, 0 to 5981
Data columns (total 4 columns):
tweet_id          5982 non-null int64
jpg_url           5982 non-null object
prediction_try    5982 non-null int64
prediction        5982 non-null object
dtypes: int64(2), object(2)
memory usage: 187.0+ KB


In [87]:
df_predict_try.sample(5)

,tweet_id,jpg_url,prediction_try,prediction
5514,793180763617361921,https://pbs.twimg.com/media/CwHxsdYVMAAqGCJ.jpg,3,airedale
1500,788765914992902144,https://pbs.twimg.com/media/CvJCabcWgAIoUxW.jpg,1,cocker spaniel
1266,749774190421639168,https://pbs.twimg.com/media/Cme7pg2XEAATMnP.jpg,1,pekinese
5961,887705289381826560,https://pbs.twimg.com/media/DFHDQBbXgAEqY7t.jpg,3,weimaraner
5071,718631497683582976,https://pbs.twimg.com/media/CfkXiX6W4AAmICF.jpg,3,persian cat


<a id="limpeza:image-prediction:conf"></a>

#### 3.1.2. [`[p1_conf, p2_conf, p3_conf]`, para `confidence`](#limpeza:image-prediction:conf)

In [88]:
# Movendo o nível de confiança das predições para uma coluna apenas
df_predict_conf = pd.melt(df_predict_clean,
                         id_vars=['tweet_id', 'jpg_url'],
                         value_vars=['p1_conf', 'p2_conf', 'p3_conf'],
                         var_name=['prediction_try'],
                         value_name='confidence'
                        )

In [89]:
df_predict_conf.sample(5)

,tweet_id,jpg_url,prediction_try,confidence
613,680176173301628928,https://pbs.twimg.com/media/CXB4nWnWEAAhLTX.jpg,p1_conf,0.207547
4515,676811746707918848,https://pbs.twimg.com/media/CWSEsO9WwAAX-fZ.jpg,p3_conf,0.060331
201,669749430875258880,https://pbs.twimg.com/media/CUttjYtWcAAdPgI.jpg,p1_conf,0.245794
1810,847116187444137987,https://pbs.twimg.com/media/C8GPrNDW4AAkLde.jpg,p1_conf,0.128935
2051,667073648344346624,https://pbs.twimg.com/media/CUHr8WbWEAEBPgf.jpg,p2_conf,0.092494


In [90]:
# Atribuir apenas valores inteiros aos indicadores de tentativas
df_predict_conf.prediction_try = df_predict_conf.prediction_try.apply(get_prediction_index)

In [91]:
df_predict_conf.sample(5)

,tweet_id,jpg_url,prediction_try,confidence
4127,668567822092664832,https://pbs.twimg.com/media/CUc64knWoAkZt70.jpg,3,0.003053
5260,750056684286914561,https://pbs.twimg.com/media/Cmfx2oNW8AAGg4H.jpg,3,0.077004
3971,888917238123831296,https://pbs.twimg.com/media/DFYRgsOUQAARGhO.jpg,2,0.120184
1434,776477788987613185,https://pbs.twimg.com/media/CsaaaaxWgAEfzM7.jpg,1,0.884839
4607,680494726643068929,https://pbs.twimg.com/media/CXGaVxOWAAADjhF.jpg,3,0.064061


In [92]:
df_predict_conf.shape

(5982, 4)

<a id="limpeza:image-prediction:dog"></a>

#### 3.1.3. [`[p1_dog, p2_dog, p3_dog]`, para `is_a_dog`](#limpeza:image-prediction:dog)

In [93]:
# Movendo o resultado da predição, `p#_dog` para `is_a_dog`
df_predict_dog = pd.melt(df_predict_clean,
                        id_vars=['tweet_id', 'jpg_url'],
                        value_vars=['p1_dog', 'p2_dog', 'p3_dog'],
                        var_name=['prediction_try'],
                        value_name='is_a_dog')

In [94]:
df_predict_dog.sample(5)

,tweet_id,jpg_url,prediction_try,is_a_dog
2549,677687604918272002,https://pbs.twimg.com/media/CWehRdEWIAAySyO.jpg,p2_dog,False
592,679503373272485890,https://pbs.twimg.com/media/CW4UtmYWsAAEjqA.jpg,p1_dog,False
2934,704113298707505153,https://pbs.twimg.com/media/CcWDTerUAAALORn.jpg,p2_dog,False
3101,724004602748780546,https://pbs.twimg.com/media/CgwuWCeW4AAsgbD.jpg,p2_dog,True
1057,715009755312439296,https://pbs.twimg.com/media/Cew5kyOWsAA8Y_o.jpg,p1_dog,False


In [95]:
# Atribuir apenas valores inteiros aos indicadores de tentativas
df_predict_dog.prediction_try = df_predict_dog.prediction_try.apply(get_prediction_index)

In [96]:
df_predict_dog.sample(5)

,tweet_id,jpg_url,prediction_try,is_a_dog
5691,826598365270007810,https://pbs.twimg.com/media/C3iq0EEXUAAdBYC.jpg,3,False
5752,836260088725786625,https://pbs.twimg.com/media/C5r-G2IUwAA6KBY.jpg,3,True
3738,832757312314028032,https://pbs.twimg.com/media/C46MWnFVYAUg1RK.jpg,2,True
4473,675501075957489664,https://pbs.twimg.com/media/CV_cnjHWUAADc-c.jpg,3,False
4801,692752401762250755,https://pbs.twimg.com/tweet_video_thumb/CZ0mhd...,3,True


In [97]:
df_predict_dog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5982 entries, 0 to 5981
Data columns (total 4 columns):
tweet_id          5982 non-null int64
jpg_url           5982 non-null object
prediction_try    5982 non-null int64
is_a_dog          5982 non-null bool
dtypes: bool(1), int64(2), object(1)
memory usage: 146.1+ KB


<a id="limpeza:image-prediction:merge"></a>

#### 3.2. [Unir os dataframes de prediction](#limpeza:image-prediction:merge)

In [98]:
# Merge
df_predict_clean = pd.merge(df_predict_try, df_predict_conf, how='left', on=['tweet_id', 'jpg_url', 'prediction_try'])

In [99]:
df_predict_clean.shape

(5982, 5)

In [100]:
df_predict_clean.sample(5)

,tweet_id,jpg_url,prediction_try,prediction,confidence
3605,811744202451197953,https://pbs.twimg.com/media/C0PlCQjXAAA9TIh.jpg,2,labrador retriever,0.202862
5412,774757898236878852,https://pbs.twimg.com/media/CsB-MYiXgAEQU20.jpg,3,lakeland terrier,0.007008
5754,836677758902222849,https://pbs.twimg.com/media/C5x57-TWUAEawQh.jpg,3,snow leopard,0.079694
5410,774314403806253056,https://pbs.twimg.com/media/Cr7q1VxWIAA5Nm7.jpg,3,saluki,0.036325
325,671896809300709376,https://pbs.twimg.com/media/CVMOlMiWwAA4Yxl.jpg,1,chow,0.243529


In [101]:
df_predict_clean = pd.merge(df_predict_clean, df_predict_dog, how='left', on=['tweet_id', 'jpg_url', 'prediction_try'])

In [102]:
df_predict_clean.shape

(5982, 6)

**Teste: Verificar Merge dos dataframes de predição**

In [103]:
df_predict_clean.sample(5)

,tweet_id,jpg_url,prediction_try,prediction,confidence,is_a_dog
220,670290420111441920,https://pbs.twimg.com/media/CU1Zgk7UcAAjw2t.jpg,1,chihuahua,0.368876,True
289,671182547775299584,https://pbs.twimg.com/media/CVCE9uYXIAEtSzR.jpg,1,rottweiler,0.331179,True
5066,718246886998687744,https://pbs.twimg.com/media/Cfe5tLWXEAIaoFO.jpg,3,siberian husky,0.057498,True
3610,812709060537683968,https://pbs.twimg.com/media/C0dSk98WEAALyya.jpg,2,golden retriever,0.182610,True
2717,686050296934563840,https://pbs.twimg.com/media/CYVXBb9WsAAwL3p.jpg,2,keeshond,0.004083,True


------
<a id="limpeza:twitter-api"></a>

## Limpeza: Twitter API

In [104]:
# Cópia do dataframe para limpeza
df_api_clean = df_tweets_api.copy()

In [105]:

df_api_clean.sample(3)

,id,created_at,in_reply_to_status_id,in_reply_to_user_id,is_quote_status,retweet_count,favorite_count,media_url,retweeted,favorited
918,700062718104104960,Wed Feb 17 21:02:13 +0000 2016,None,None,False,731,2754,http://pbs.twimg.com/media/CbcfUxoUAAAlHGK.jpg,False,False
163,816829038950027264,Thu Jan 05 02:09:53 +0000 2017,None,None,False,5103,0,http://pbs.twimg.com/media/CvoBPWRWgAA4het.jpg,False,False
1804,666099513787052032,Mon Nov 16 03:44:34 +0000 2015,None,None,False,66,150,http://pbs.twimg.com/media/CT51-JJUEAA6hV8.jpg,False,False


<a id="limpeza:tweets-api:columns"></a>

### 1. [Os nomes das colunas `id` e `media_url` estão fora do padrão dos demais dataframes, respectivamente `tweet_id` e `jpg_url` ](#limpeza:tweets-api:columns)

In [106]:
df_api_clean.rename(columns={'id':'tweet_id', 'media_url': 'jpg_url'}, inplace=True)

**Teste: Verificar colunas renomeadas**

In [107]:
df_api_clean.columns

Index(['tweet_id', 'created_at', 'in_reply_to_status_id',
       'in_reply_to_user_id', 'is_quote_status', 'retweet_count',
       'favorite_count', 'jpg_url', 'retweeted', 'favorited'],
      dtype='object')

<a id="limpeza:tweets-api:duplicated"></a>

### 2. [Diferentes Tweets referenciando a mesma imagem, duplicidade](#limpeza:tweets-api:duplicated)

Assim como na [limpeza de duplicidade de imagens no dataframe de predições](#limpeza:image-prediction:duplicated-urls), é importante de remover os tweets não originais, os retweets. Com auxílio do dataframe `df_arch_clean`, que possui somente referências a tweets originais, será possível identifica-los e descartar os tweets indesejados.

In [108]:
df_api_clean.shape

(1820, 10)

In [109]:
df_api_clean = df_api_clean.loc[df_api_clean.tweet_id.isin(df_arch_clean.tweet_id)]

In [110]:
df_api_clean.shape

(1746, 10)

Ao remover os ids que referenciavam a **retweets**, houve uma redução de **74** registros, de **1820** para **1746**

<a id="limpeza:tweets-api:datetime"></a>

### 3. [Data de criação (`created_at`) como `string`](#limpeza:tweets-api:datetime)
* Converter datas em string para o formato `datetime`

In [111]:
df_api_clean.created_at = pd.to_datetime(df_api_clean.created_at)

**Teste: Validar tipo de variável em `created_at`**

In [112]:
df_api_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1746 entries, 0 to 1819
Data columns (total 10 columns):
tweet_id                 1746 non-null int64
created_at               1746 non-null datetime64[ns, UTC]
in_reply_to_status_id    1746 non-null object
in_reply_to_user_id      1746 non-null object
is_quote_status          1746 non-null bool
retweet_count            1746 non-null int64
favorite_count           1746 non-null int64
jpg_url                  1746 non-null object
retweeted                1746 non-null bool
favorited                1746 non-null bool
dtypes: bool(3), datetime64[ns, UTC](1), int64(3), object(3)
memory usage: 114.2+ KB


In [113]:
df_api_clean.sample(5)

,tweet_id,created_at,in_reply_to_status_id,in_reply_to_user_id,is_quote_status,retweet_count,favorite_count,jpg_url,retweeted,favorited
20,882762694511734784,2017-07-06 00:46:41+00:00,None,None,False,4759,27617,http://pbs.twimg.com/media/DEAz_HHXsAA-p_z.jpg,False,False
1444,672997845381865473,2015-12-05 04:36:04+00:00,None,None,False,735,1984,http://pbs.twimg.com/media/CVb39_1XIAAMoIv.jpg,False,False
1512,671533943490011136,2015-12-01 03:39:03+00:00,None,None,False,590,1031,http://pbs.twimg.com/media/CVHEju0XAAEUZRY.jpg,False,False
1682,668528771708952576,2015-11-22 20:37:34+00:00,None,None,False,232,463,http://pbs.twimg.com/media/CUcXXpxWUAAUJ__.jpg,False,False
1041,689905486972461056,2016-01-20 20:21:00+00:00,None,None,False,731,2524,http://pbs.twimg.com/media/CZMJYAkUMAATdcm.jpg,False,False


<a id="limpeza:tweets-api:info"></a>

### 4. [Valor do id na coluna `in_reply_to_status_id` e `in_reply_to_user_id` como `string`](#limpeza:tweets-api:info)
* Converter para Inteiro as colunas `in_reply_to_status_id` e `in_reply_to_user_id` e atribuir np.nan as que estiverem sem valor

In [114]:
df_api_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1746 entries, 0 to 1819
Data columns (total 10 columns):
tweet_id                 1746 non-null int64
created_at               1746 non-null datetime64[ns, UTC]
in_reply_to_status_id    1746 non-null object
in_reply_to_user_id      1746 non-null object
is_quote_status          1746 non-null bool
retweet_count            1746 non-null int64
favorite_count           1746 non-null int64
jpg_url                  1746 non-null object
retweeted                1746 non-null bool
favorited                1746 non-null bool
dtypes: bool(3), datetime64[ns, UTC](1), int64(3), object(3)
memory usage: 114.2+ KB


Converter os valores em `None` para np.nan em `in_reply_to_status_id` e `in_reply_to_user_id`

In [115]:
def set_zero_at_none(value):
    if str(value).lower() == 'none':
        return '0'
    else:
        return value

In [116]:
df_api_clean.in_reply_to_status_id = df_api_clean.in_reply_to_status_id.apply(set_zero_at_none)

In [117]:
df_api_clean.in_reply_to_user_id = df_api_clean.in_reply_to_user_id.apply(set_zero_at_none)

In [118]:
df_api_clean.loc[:, ['in_reply_to_user_id', 'in_reply_to_status_id']] = df_api_clean.loc[:, ['in_reply_to_user_id', 'in_reply_to_status_id']].astype('int64')

Listar exemplos com valores 'None'substituídos por '0' e convertidos para `int64`

In [119]:
df_api_clean.loc[:, ['in_reply_to_user_id', 'in_reply_to_status_id']].sample(5)

,in_reply_to_user_id,in_reply_to_status_id
39,0,0
1098,0,0
782,0,0
1056,0,0
885,4196983835,703041949650034688


Converter valores iguais a 0 por `np.nan`

In [120]:
# Função para conversão de np.nan para 0
set_nan_at_zero = lambda value: np.nan if value is 0 else value

In [121]:
df_api_clean.in_reply_to_status_id = df_api_clean.in_reply_to_status_id.apply(set_nan_at_zero)

In [122]:
df_api_clean.in_reply_to_user_id = df_api_clean.in_reply_to_user_id.apply(set_nan_at_zero)

**Teste: Confirmar conversão para inteiro das colunas `in_reply_to_status_id` e `in_reply_to_user_id`**

In [123]:
df_api_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1746 entries, 0 to 1819
Data columns (total 10 columns):
tweet_id                 1746 non-null int64
created_at               1746 non-null datetime64[ns, UTC]
in_reply_to_status_id    22 non-null float64
in_reply_to_user_id      22 non-null float64
is_quote_status          1746 non-null bool
retweet_count            1746 non-null int64
favorite_count           1746 non-null int64
jpg_url                  1746 non-null object
retweeted                1746 non-null bool
favorited                1746 non-null bool
dtypes: bool(3), datetime64[ns, UTC](1), float64(2), int64(3), object(1)
memory usage: 114.2+ KB


# Dataframe principal

In [124]:
df_arch_clean.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,text,rating_numerator,rating_denominator,name,category
1794,677328882937298944,NaN,NaN,2015-12-17 03:26:04+00:00,This is Derek. All the dogs adore Derek. He's ...,10,10,Derek,NaN
1292,708109389455101952,NaN,NaN,2016-03-11 01:56:49+00:00,This is Remington. He was caught off guard by ...,10,10,Remington,NaN
15,889278841981685760,NaN,NaN,2017-07-24 00:19:32+00:00,This is Oliver. You're witnessing one of his m...,13,10,Oliver,NaN
1843,675853064436391936,NaN,NaN,2015-12-13 01:41:41+00:00,Here we have an entire platoon of puppers. Tot...,88,80,NaN,NaN
1166,722613351520608256,NaN,NaN,2016-04-20 02:30:23+00:00,Meet Schnitzel. He's a Tropicana Floofboop. Ge...,12,10,Schnitzel,NaN


In [125]:
df_api_clean.sample(5)

,tweet_id,created_at,in_reply_to_status_id,in_reply_to_user_id,is_quote_status,retweet_count,favorite_count,jpg_url,retweeted,favorited
1642,669214165781868544,2015-11-24 18:01:05+00:00,NaN,NaN,False,170,441,http://pbs.twimg.com/media/CUmGu7-UcAA0r3O.jpg,False,False
1386,674318007229923329,2015-12-08 20:01:55+00:00,NaN,NaN,False,529,1436,http://pbs.twimg.com/media/CVuopr8WwAExw_T.jpg,False,False
90,837110210464448512,2017-03-02 01:20:01+00:00,NaN,NaN,False,2539,16681,http://pbs.twimg.com/media/C54DS1kXQAEU5pS.jpg,False,False
1240,678764513869611008,2015-12-21 02:30:45+00:00,NaN,NaN,False,503,1694,http://pbs.twimg.com/media/CWt0ubZWcAAkFER.jpg,False,False
624,740676976021798912,2016-06-08 22:48:46+00:00,NaN,NaN,False,7176,18874,http://pbs.twimg.com/media/Ckdpx5KWsAANF6b.jpg,False,False


In [127]:
df_twitter_master = pd.merge(df_arch_clean, df_api_clean.loc[:, ['tweet_id', 'retweet_count', 'favorite_count']], how='left', on=['tweet_id'])

In [128]:
df_twitter_master.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,text,rating_numerator,rating_denominator,name,category,retweet_count,favorite_count
1619,677228873407442944,NaN,NaN,2015-12-16 20:48:40+00:00,Say hello to Chuq. He just wants to fit in. 11...,11,10,Chuq,NaN,1719.0,3642.0
2046,668286279830867968,NaN,NaN,2015-11-22 04:33:59+00:00,Meet Rusty. Rusty's dreaming of a world where ...,11,10,Rusty,NaN,139.0,506.0
687,765719909049503744,NaN,NaN,2016-08-17 01:20:27+00:00,This is Brat. He has a hard time being ferocio...,12,10,Brat,NaN,2302.0,7616.0
1463,684097758874210310,NaN,NaN,2016-01-04 19:43:10+00:00,Say hello to Lupe. This is how she sleeps. 10/...,10,10,Lupe,NaN,1523.0,4272.0
1732,674410619106390016,NaN,NaN,2015-12-09 02:09:56+00:00,This is Lenny. He wants to be a sprinkler. 10/...,10,10,Lenny,NaN,481.0,1218.0


------
<a id="armazenamento"></a>

# Armazenamento

Salvar os dataframes limpos

In [129]:
data_folder_path = 'data'

In [130]:
# Dataframe clean: Twitter Archive
df_arch_clean.to_csv(os.path.join(data_folder_path, 'twitter_archive_clean.csv'), index=None, header=True)

In [131]:
# Dataframe clean: Tweet image prediction
df_predict_clean.to_csv(os.path.join(data_folder_path, 'tweet_image_prediction_clean.csv'), index=None, header=True)

In [132]:
# Dataframe clean: API requested Tweets
df_api_clean.to_csv(os.path.join(data_folder_path, 'twitter_api_clean.csv'), index=None, header=True)

In [133]:
# Dataframe Principal: Twitter Master 
df_twitter_master.to_csv(os.path.join(data_folder_path, 'twitter_archive_master.csv'), index=None, header=True)

# Relatórios

* Data wrangling efforts
* Analyses and visualizations